## Packages:

## Instalação:

## Importações:

In [ ]:
# Importações para automação web:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Importação pra trabalhar com o sistema
import os
import sys

# Importação para barra de progresso:
from tqdm import tqdm

# Importação administrar os downloads:
import time

## Download:

In [ ]:
class CalipsoDownloader:
    '''
        A classe Downloader foi construida para lidar com os donwloads
    dos arquivos HDF para os dois casos que utilizamos, onde o donwload dos
    dados de Level 2 é estruturado de uma forma diferente do download dos 
    dados de Level 3.
    '''
    def __init__(self, folder_path, page):
        self.folder_path = folder_path
        self.page = page
        self.opcoes = Options()
        self.opcoes.add_experimental_option("prefs", {
          "download.default_directory": self.folder_path,
          "download.prompt_for_download": False,
          "download.directory_upgrade": True,
          "safebrowsing.enabled": True
        })
        self.servico = Service(ChromeDriverManager().install())
        self.navegador = webdriver.Chrome(options=self.opcoes,service=self.servico)
    
    def downloadL2(self):
        try:
            self.navegador.get(self.page)
        except:
            print('Ih rapaz... Inseriu o url corretamente?')
        
        elements = self.navegador.find_elements('xpath',"/html/body/table/tbody/tr[*]/td[2]/a")
        hrefs = [element.get_attribute("href") for element in elements]
        for href in tqdm(hrefs):
            if href.endswith(".hdf"):
                self.navegador.get(href)
            time.sleep(1)
        time.sleep(3)
        self.navegador.quit()
        print('Tudo Baixado')
        
    def downloadL3(self):
        email = input('Insira seu email:\n')
        senha = input('Insira sua senha:\n')
        self.navegador.get(self.page)
        self.navegador.find_element('xpath','//*[@id="username"]').send_keys(email)
        self.navegador.find_element('xpath','//*[@id="password"]').send_keys(senha)
        self.navegador.find_element('xpath','//*[@id="login"]/p[8]/input').click()
        time.sleep(25)
        expand_botton = self.navegador.find_element('xpath',
                                           '//*[@id="app"]/div/div/div/div/section/div/div/div[2]/ul/li/div/div[2]/div/div[1]/div[2]/header/button[3]/span')
        expand_botton.click()
        elements = self.navegador.find_elements('xpath',"/html/body/div[7]/div/div/div[2]/div/div/ul/li[*]/a")
        hrefs = [element.get_attribute("href") for element in elements]
        for href in tqdm(hrefs):
            try:
                self.navegador.get(href)
            except WebDriverException:
                print("Erro ao acessar o link, tentando novamente em 5 segundos...")
                time.sleep(5)
                self.navegador.get(href)
            time.sleep(1)
        time.sleep(3)
        self.navegador.quit()
        print('Tudo Baixado')

In [ ]:
# Input para definição do tipo de download: 
print('''
    Vale ressaltar que a forma de se baixar os dados do L2 servem
para o L1 também, ou seja, L2 como resposta também serve como resposta para L1
''')
escolha = input('Qual tipo de dado gostaria de baixar? (L2/L3)')

# Checagem da entrada:
if escolha != "L2" and escolha != "L3":
    print('Essa é uma escolha binária, ou é L2 ou L3')
    sys.exit()
    
#inputs:
diretorio = input('Digite o diretório para os arquivos\n')
pasta = input('Digite a pasta onde ficara esse arquivo') #Recomendação: Digite um nome significativo.
pagina = input('Insira a url da página que se queria acessar\n')

'''
    OBS: caso queira colocar um valor padrão para as variáveis de entrada (inputs),
pode montar simplesmente uma estrutura de condicional:
if diretorio == '':
    diretorio = #Nome do seu diretório
    
Note:
    O diretorio nesse contexto é o caminho até a pasta e a pasta é onde ficaram os arquivos,
poderia ser tudo junto, mas com grandes quantidades de arquivos, ficaria bagunçado. Por uma questão
de organização, prefiri dividir, inclusive já para criar em quem for utilizar o código esse costume,
de usar pastas diferentes e não embolar os arquivos tudo junto.
'''
# Configurações da Pasta
folder_path = os.path.join(diretorio, pasta)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)


# Instanciando a classe correspondente à escolha do usuário
if escolha == "L2":
    downloader = CalipsoDownloader(folder_path, pagina).downloadL2()
    
if escolha == "L3":
    downloader = CalipsoDownloader(folder_path, pagina).downloadL3()